# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import json
import os

# Import API key
from api_keys import g_key

In [2]:
#Load the csv exported in Part I to a DataFrame
file_to_load = "../output_data/weather.csv"
city_data = pd.read_csv(file_to_load)
city_data.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed
0,0,Hermanus,0,ZA,1635141600,70,-34.4187,19.2345,58.08,13.02
1,1,Chokurdakh,100,RU,1635141600,99,70.6333,147.9167,-4.72,3.02
2,2,Mataura,97,NZ,1635141600,78,-46.1927,168.8643,60.85,2.71
3,3,Bontang,100,ID,1635141600,73,0.1333,117.5000,83.41,10.49
4,4,São Geraldo do Araguaia,68,BR,1635141600,92,-6.4006,-48.5550,74.19,0.56


In [3]:
#city_data.shape

In [4]:
city_df = city_data.dropna()
city_df.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed
0,0,Hermanus,0,ZA,1635141600,70,-34.4187,19.2345,58.08,13.02
1,1,Chokurdakh,100,RU,1635141600,99,70.6333,147.9167,-4.72,3.02
2,2,Mataura,97,NZ,1635141600,78,-46.1927,168.8643,60.85,2.71
3,3,Bontang,100,ID,1635141600,73,0.1333,117.5000,83.41,10.49
4,4,São Geraldo do Araguaia,68,BR,1635141600,92,-6.4006,-48.5550,74.19,0.56


In [5]:
#city_df.shape

In [6]:
#Configure gmaps
gmaps.configure(api_key=g_key)

In [7]:
#Use the Lat and Lng as locations
location_df = city_df[["Latitude", "Longitude"]].astype(float)

# Use humidity as the weight
humidity_df = city_df["Humidity"].astype(float)

In [ ]:
#Add Heatmap layer to map
fig = gmaps.figure(center =[0,0], zoom_level = 3)

heat_layer = gmaps.heatmap_layer(location_df, weights = humidity_df, 
                                 dissipating = False, 
                                 max_intensity = 100, 
                                 point_radius = 1 )
fig.add_layer(heat_layer)
fig

In [9]:
vocation_cities = pd.DataFrame()

for index, row in city_df.iterrows():
    if row["Max Temp"]>70 and row["Max Temp"]<80 and row["Wind Speed"]< 10 and row["Cloudiness"]==0:
        vocation_cities[index] = row
        
clean_df = vocation_cities.T

In [10]:
del clean_df["Unnamed: 0"]

In [11]:
reset_df = clean_df.reset_index()

In [12]:
reset_df

,index,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed
0,149,Abu Kamal,0,SY,1635141600,31,34.4506,40.9171,73.27,5.37
1,230,Port Said,0,EG,1635141600,68,31.2565,32.2841,72.55,3.36
2,247,San Patricio,0,US,1635141600,88,28.017,-97.5169,78.22,6.51
3,255,Kurali,0,IN,1635141600,32,30.8315,76.5785,79.66,5.86
4,287,Shanghai,0,CN,1635141600,53,31.2222,121.4581,70.25,7.9
5,387,Aswān,0,EG,1635141600,48,24.0934,32.907,74.07,8.86
6,480,Matamoros,0,MX,1635141600,31,25.5333,-103.25,70.63,0.74
7,481,Álamos,0,MX,1635141600,76,27.0167,-108.9333,74.71,3.56


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [13]:
#Store into variable named hotel_df
hotel_df =reset_df
hotel_df

,index,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed
0,149,Abu Kamal,0,SY,1635141600,31,34.4506,40.9171,73.27,5.37
1,230,Port Said,0,EG,1635141600,68,31.2565,32.2841,72.55,3.36
2,247,San Patricio,0,US,1635141600,88,28.017,-97.5169,78.22,6.51
3,255,Kurali,0,IN,1635141600,32,30.8315,76.5785,79.66,5.86
4,287,Shanghai,0,CN,1635141600,53,31.2222,121.4581,70.25,7.9
5,387,Aswān,0,EG,1635141600,48,24.0934,32.907,74.07,8.86
6,480,Matamoros,0,MX,1635141600,31,25.5333,-103.25,70.63,0.74
7,481,Álamos,0,MX,1635141600,76,27.0167,-108.9333,74.71,3.56


In [14]:
#add a new column as "Hotel Name"to the current dataframe     
hotel_df["Hotel Name"] = ""

params = {"radius": 50000, "types": "hotel", "keyword": "hotels", "key": g_key}

# Use the lat/lng to identify hotels
for index, row in hotel_df.iterrows():
    
    # get lat, lng 
    lat = row["Latitude"]
    lng = row["Longitude"]
    
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
  
    hotel_name = requests.get(base_url, params=params)
    
    # convert to json
    hotel_name = hotel_name.json()
    print(json.dumps(hotel_name, indent=4, sort_keys=True))
    
    # not to crush the code if no data found
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
hotel_df      
        

{
    "html_attributions": [],
    "results": [],
    "status": "ZERO_RESULTS"
}
Missing field/result... skipping.
{
    "html_attributions": [],
    "next_page_token": "Aap_uECqkblivg1NfTknk7VODqIgCee0v6Nhd8dl1Lvw_ZfQ9m2nr99Du1k0WK0sPwG40oac5hus_plXgarPISYb2ZhSjIYSouFQXFyBm-_eqY5bAK4Ao5AtC8Qn1rRYdAvyB8cMP-hx8Uue4X8oHF1Pn9KPqcL7rcbDimqPvu6vk8VOneW_-ARj5aKGk_PESFJpxfTyumwO91bepwzgy0ykm14VzwduzzyyY8Q6l3alTzu8OAqAV7z1fJjcgHyqmDDJQtDb_Bgn_u7RiwFxvuW1TWDCntNicwsyCogferI3PZAByDmRicsTNDomTj8YOX1b7WLjh2_oHcKTckflOG1j0lWg_YuZfTi8aMLDpHpoKJsKSJf7lnDMSLI_-aWIBICOKw6TfWzdLAKv4YIvazvi6HBFOVYJWwKVL9Oh4DDo9zAP3NyWCvwnH1eiA3tLQyRQ",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 31.5245893,
                    "lng": 31.8425528
                },
                "viewport": {
                    "northeast": {
                        "lat": 31.52584007989273,
                        "lng": 31.8

{
    "html_attributions": [],
    "next_page_token": "Aap_uEAJCveEh7_pQneADpza-S_zb_XlZh8MRGISWrCyjiA_KfClULs7Knxz-iWlHMqJMIba33Jt5p3iIdY-sU_d-0_CG4kuyKqbEithKNYG6z8e9FSeIcxHkGZ5WxY5f-I4WFCGWgX7QLhDTtiF0kg1XdR4qxr0aaa4oPNcIxrHEMMvdzQg-qEgKgGWvvJuiOK0vgSyRTaBE-gZaDv4Opfhqz7OyhrHFtOBwnr5uLxEdRmAisEwkeiqMvP-eFf8efTs5f8iwrwujh1kM5OKtQYQHiEUdir64yYoT-74svFmELGBWCqKjISc9WP3dLRqRRoOaSdsFIed2Hv3jT5s0VhJ46O3lUgOCgAF_BtlzUnKF6zjBjBiG4TFy87sw8Qw5GIqc-cYuFHtYYxJ0597oy2l75OZvJoSAZq9x1qUTPbPWZcC22_56X1zSHL6T4ZgNfB_",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 27.7815408,
                    "lng": -97.3942486
                },
                "viewport": {
                    "northeast": {
                        "lat": 27.78291042989272,
                        "lng": -97.39302872010728
                    },
                    "southwest": {
                        "lat": 27.7802107

{
    "html_attributions": [],
    "next_page_token": "Aap_uECH9ozIlrFlsdBtao89k2lbvj6iK8pJIdiM21ITtGuEvDO03MXi-HqL8C9Apu7pFtnoh5LzI-Ed34hyd2GlULUuGN_--41EZDGRb6XWr-g0WEf-rjLFrSyfVsHBhHTVrg_1ScEy7SN7rcGILB94ZTJPvi589t0QrBD3xk4Iq3Y3Gz51D_l3f2qn6eBxlM9QxRqkLZDcLq0bvXVl6iK2muiR5NCGtKrVrD7WKIPHr0qoAjE5j5-xjN0vwDJr6Ayy03tkkDArMetozfKsEfGDGiMCldKBpnlu3geK3Qax-e-nPh92RQSBx7G1eQDIik4DTw8_5bPzN6WWAc34pSseogyf2AKKz8bSCpBhfbjoZvzx6kVUFD2HePDYpeh2j0PElm1cUnjYQ4nmG8He1cHOw9LLc_hjREacZvfrxI6JBYgwoNFg4NKJAVRVRJfVQcoT",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 30.6890809,
                    "lng": 76.8715093
                },
                "viewport": {
                    "northeast": {
                        "lat": 30.69042392989272,
                        "lng": 76.87289092989272
                    },
                    "southwest": {
                        "lat": 30.687724270

{
    "html_attributions": [],
    "next_page_token": "Aap_uEDvIFG8phUCiRO8YX_ugBlR-p9dnwoIlBT7aBYL9ug7DOhlr-q93Nqi8w7vcmxGQUJ4p6jZRP_6gEorRtnXoQ9nJrqWc6vwFIzD7rbSN7nm71B1bFHjhZ72kGcDqNYc9j6A33kxsoyOOtnCmV7Cd1LDWNx_K1pL6hsxmqnOF1Kg4OzirQnLq6Xyqa_z5BKOE5sqZhCDGYZhBXm4wMVpmylS7rU8NrXMVxeZs71iKGvNcrPsBwdOElyWLEZpl2nuiIxIDjqse7Tv6DX9bR1nRmK0dcVH2tWYYFcFgMhuQjtRQk9hRXlpAFzJX0czSiBgoThOe9ssyd13t_cMzdwyftOyMRNocZ1Aervdbw_h7PmQX-CTro07BcU9HG-3TWB61pn7-Jl2xNxz_7REMMKZOomvze6syEjlfADcoeEekSXctxb2LU_GNjgONf91G-Od",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 31.2353438,
                    "lng": 121.5057797
                },
                "viewport": {
                    "northeast": {
                        "lat": 31.23704647989272,
                        "lng": 121.5072914298927
                    },
                    "southwest": {
                        "lat": 31.23434682

{
    "html_attributions": [],
    "next_page_token": "Aap_uEAcOjEleM-nKBFVL5PqObShoKBMV58-8FcljUj5txReYgBJDzyXLp-LXT-tHk7zxY4EmjYKZ9WS9otSIsA2czLoi183oJq3-rL4rdotT4z9_cOy3hM6AfTTmti1kayqFKXiuYJNbzmJCbSTc8X4CJBaj3DMReofkOGfEmfdI6oNjihnyzhoJRiFykr6UhDlodC6hJ6cHA3nQAQDXx9ICYrkDXTN0ymy4IRm1E6sv-3uZjK4M1mREbWBOtMEAeASv6QLDlwXlxX7rvAWhxdSHi7rIDZbQ-9mKl3ReGq0OmN8IbPgGDBklNwRmh2OL0Bax8a6ZmsAZopZpt5i5RCl3urigxvtY9SNxNigUtWW4pdrWdbZDNiRMkU1VOLUQcSXnTnHLovEt3zDRqLdDldL4M6_VdHMmqLTLkNPKLnYR_pUyAcFCbKSiDZmdJO6Z9In",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 24.0823298,
                    "lng": 32.8878313
                },
                "viewport": {
                    "northeast": {
                        "lat": 24.08371787989272,
                        "lng": 32.88923327989272
                    },
                    "southwest": {
                        "lat": 24.081018220

{
    "html_attributions": [],
    "next_page_token": "Aap_uEDFR2m03uWojz1oRKGxD832XUquRKaUbsUfkGJNEPhLp8ql1SuBMYFBQNH1z_TVq4GI5T3znMZumWimVQZ-KjOIg_GtCSGqwoyQ8RIwZ1bIKKBnqjO-40peu-Cz1ZpiW9GuX8AOy9jPMlLz7854EF5Xhkk8AFUzNl_h7frgEyxPDr9NVwFCkOAZDAHGAXAzwnj_tsCUbH6ION0NWPlbeh4Dlwhd1OOjkRLxKGbhx03eaPNh9IO15LaSKibRLywCQDh55WjRode1w6OpMcP0vtAQB7elPw5pOarGrCUgfZxZwzOrDElRNEi7aEukk3Ct9qpoUfGXjFLENiIDFaGmN4mOqlKHeuFeYyVkcYjfGc7VkyBulAmNz6dtucDWmqQoCwHelbVRWwR51A1hykkjxvN7tXaDvldGkh74_WsT_4nfS_FTLE1d9ii2AMWjr-aS",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 25.5695719,
                    "lng": -103.496451
                },
                "viewport": {
                    "northeast": {
                        "lat": 25.57088322989272,
                        "lng": -103.4951935701073
                    },
                    "southwest": {
                        "lat": 25.5681835

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 27.0451232,
                    "lng": -108.9779736
                },
                "viewport": {
                    "northeast": {
                        "lat": 27.04639532989272,
                        "lng": -108.9766696201073
                    },
                    "southwest": {
                        "lat": 27.04369567010728,
                        "lng": -108.9793692798927
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "icon_background_color": "#909CE1",
            "icon_mask_base_uri": "https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet",
            "name": "\u00c1lamos Bonito Resort",
            "opening_hours": {
                "open_now": false
 

,index,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed,Hotel Name
0,149,Abu Kamal,0,SY,1635141600,31,34.4506,40.9171,73.27,5.37,
1,230,Port Said,0,EG,1635141600,68,31.2565,32.2841,72.55,3.36,Steigenberger Hotel El Lessan
2,247,San Patricio,0,US,1635141600,88,28.017,-97.5169,78.22,6.51,"Emerald Beach Hotel Corpus Christi, TX"
3,255,Kurali,0,IN,1635141600,32,30.8315,76.5785,79.66,5.86,"Holiday Inn Chandigarh Panchkula, an IHG Hotel"
4,287,Shanghai,0,CN,1635141600,53,31.2222,121.4581,70.25,7.9,Grand Hyatt Shanghai
5,387,Aswān,0,EG,1635141600,48,24.0934,32.907,74.07,8.86,Sofitel Legend Old Cataract Aswan
6,480,Matamoros,0,MX,1635141600,31,25.5333,-103.25,70.63,0.74,Hotel La Siesta
7,481,Álamos,0,MX,1635141600,76,27.0167,-108.9333,74.71,3.56,Álamos Bonito Resort


In [15]:
hotel_df

,index,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed,Hotel Name
0,149,Abu Kamal,0,SY,1635141600,31,34.4506,40.9171,73.27,5.37,
1,230,Port Said,0,EG,1635141600,68,31.2565,32.2841,72.55,3.36,Steigenberger Hotel El Lessan
2,247,San Patricio,0,US,1635141600,88,28.017,-97.5169,78.22,6.51,"Emerald Beach Hotel Corpus Christi, TX"
3,255,Kurali,0,IN,1635141600,32,30.8315,76.5785,79.66,5.86,"Holiday Inn Chandigarh Panchkula, an IHG Hotel"
4,287,Shanghai,0,CN,1635141600,53,31.2222,121.4581,70.25,7.9,Grand Hyatt Shanghai
5,387,Aswān,0,EG,1635141600,48,24.0934,32.907,74.07,8.86,Sofitel Legend Old Cataract Aswan
6,480,Matamoros,0,MX,1635141600,31,25.5333,-103.25,70.63,0.74,Hotel La Siesta
7,481,Álamos,0,MX,1635141600,76,27.0167,-108.9333,74.71,3.56,Álamos Bonito Resort


In [16]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [17]:
# Add marker layer ontop of heat map
fig =gmaps.figure()
markers = gmaps.marker_layer(locations, hotel_info)
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))